In [4]:
# Import necessary packages

import numpy as np
import opendssdirect as dss
# from utils.device.Inverter import Inverter
# from utils.controller.AdaptiveInvController import AdaptiveInvController
# from utils.controller.FixedInvController import FixedInvController
import matplotlib.pyplot as plt
from math import tan,acos
import copy
import pandas as pd
import time

from collections import deque
import signal_processing as signal_processing

# import numpy as np
# from matplotlib.mlab import PCA
# from sklearn.decomposition import PCA

import scipy.linalg as spla

In [12]:
# Set global variables and parameters, and test OpenDSS with network


#######################################################
###Global variable initialization and error checking###
#######################################################

# Sbase=1
# LoadScalingFactor = 3
# GenerationScalingFactor = 5

# NoiseMultiplyer= 1
# #Set simulation analysis period - the simulation is from StartTime to EndTime
# StartTime = 40600
# EndTime = StartTime + 640
# EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
# #Set hack parameters
# TimeStepOfHack = 160
# PercentHacked = np.array([0,0,0,0,0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0])

# #Set initial VBP parameters for uncompromised inverters
# VBP_normal=np.array([0.97, 1, 1, 1.03, 1.06])
# VBP_attack=np.array([0.997, 1, 1, 1.003, 1.007])

# #Set delays for each node
# Delay_VBPCurveShift = (30+2*np.random.randn(31)).astype(int)
# #Delay_VBPCurveShift = (10+2*np.random.randn(31)).astype(int)
# lpf_meas_vector = (1+0.2*np.random.randn(31))
# lpf_output_vector= (0.1+0.015*np.random.randn(31))

# #Set observer voltage threshold
# ThreshHold_vqvp = 0.06
# adaptive_gain=800


# power_factor=0.9
# pf_converted=tan(acos(power_factor))
# # Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters




# #Error checking of the global variable -- TODO: add error handling here!
# if EndTime < StartTime or EndTime < 0 or StartTime < 0:
#     print('Setup Simulation Times Inappropriately.')
# if NoiseMultiplyer < 0:
#     print('Setup Noise Multiplyer Correctly.')

# Select OpenDSS file
# dss.run_command('Redirect IEEE_13_Node/IEEE_13_Bus_original.dss')
# dss.run_command('Redirect IEEE_13_Node/IEEE_13_Bus_allwye.dss')
dss.run_command('Redirect IEEE_13_Node/IEEE_13_Bus_allwye_noxfm_noreg.dss')

# Set slack bus (sourcebus) voltage reference in p.u.
SlackBusVoltage = 1.00
dss.Vsources.PU(SlackBusVoltage)

# Solve power flow with OpenDSS file
dss.Solution.Solve()
if not dss.Solution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    dss.Monitors.ResetAll()
    
    #set solution Params
    #setSolutionParams(dss,'daily',1,1,'off',1000000,30000)
    dss.Solution.Mode(1)
    dss.Solution.Number(1)
    dss.Solution.StepSize(1)
    dss.Solution.ControlMode(-1)
    dss.Solution.MaxControlIterations(1000000)
    dss.Solution.MaxIterations(30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=dss.Loads.Count()
    AllLoadNames=dss.Loads.AllNames()
    print('OpenDSS Model Compliation Done.')

print('')

# print(dss.Solution.Converged())

# Print number of buses, and bus names
print(len(dss.Circuit.AllBusNames()))
print(dss.Circuit.AllBusNames())

# Print number of loads, and load names
print(len(dss.Loads.AllNames()))
print(dss.Loads.AllNames())

print('')

# Print different voltage measurements
for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('')
        
# Print different voltage measurements
for k1 in range(len(dss.Circuit.AllBusNames())):
    if dss.Circuit.AllBusNames()[k1] == 'sourcebus':
        dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
        source_voltage = dss.Bus.PuVoltage()
        print(dss.Circuit.AllBusNames()[k1])
        print('BASE - LN')
        print(dss.Bus.kVBase())
        print('VMAGANGLE - LN POLAR')
        print(dss.Bus.VMagAngle())
        print('puVMAGANGLE - LN POLAR')
        print(dss.Bus.puVmagAngle())
        print('VOTLAGES - LN CARTESIAN')
        print(dss.Bus.Voltages())
        print('puVOTLAGES - LN CARTESIAN')
        print(dss.Bus.PuVoltage())
        print('VLL - LL CARTESIAN')
        print(dss.Bus.VLL())
        print('puVLL - LL CARTESIAN')
        print(dss.Bus.puVLL())
        
        tempvoltage = dss.Bus.Voltages()
        
        Vab = tempvoltage[0]
        
print('')
        
for k1 in range(len(dss.Circuit.AllBusNames())):
    dss.Circuit.SetActiveBus(dss.Circuit.AllBusNames()[k1])
    print(dss.Circuit.AllBusNames()[k1])
    print(dss.Bus.Nodes())
    print('BASE - LN')
    print(dss.Bus.kVBase())
    print('VMAGANGLE - LN POLAR')
    print(dss.Bus.VMagAngle())
    print('puVMAGANGLE - LN POLAR')
    print(dss.Bus.puVmagAngle())
    print('')
    
for k1 in range(len(dss.Lines.AllNames())):
    dss.Lines.Name(dss.Lines.AllNames()[k1])
    print(dss.Lines.AllNames()[k1])
    print(dss.Lines.Bus1(), ' - ', dss.Lines.Bus2())
    print(dss.Lines.Length())
    print(dss.CktElement.Powers())
    Sk = np.asarray(dss.CktElement.Powers())[::2] + 1j*np.asarray(dss.CktElement.Powers())[1::2]
    print(Sk)
    # print(dss.CktElement.YPrim())
    # print(dss.Lines.Phases())
    # YPrim = np.asarray(dss.CktElement.YPrim())
    # YPrim = YPrim[0::2] + 1j*YPrim[1::2]
    # YPrim = YPrim.reshape(2*dss.Lines.Phases(),2*dss.Lines.Phases())
    # print(YPrim)
    # YPrim = YPrim[0:dss.Lines.Phases(),0:dss.Lines.Phases()]
    # print(YPrim)
    print('')

for k1 in range(len(dss.Loads.AllNames())):
    dss.Loads.Name(dss.Loads.AllNames()[k1])
    print(dss.Loads.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.Capacitors.AllNames())):
    print(dss.Capacitors.AllNames()[k1])
    dss.Capacitors.Name(dss.Capacitors.AllNames()[k1])
    print(dss.CktElement.BusNames())
    print(dss.CktElement.NodeOrder())
    print(dss.CktElement.Powers())
    print('')

for k1 in range(len(dss.RegControls.AllNames())):
    print(dss.RegControls.AllNames()[k1])
    dss.RegControls.Name(dss.RegControls.AllNames()[k1])
    print(dss.RegControls.TapNumber())
    print('')


Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.

16
['sourcebus', '650', 'rg60', '633', '634', '671', '645', '646', '692', '675', '611', '652', '670', '632', '680', '684']
15
['671', '634a', '634b', '634c', '645', '646', '692', '675a', '675b', '675c', '611', '652', '670a', '670b', '670c']

sourcebus
BASE - LN
66.39528095680697
VMAGANGLE - LN POLAR
[66385.62722334421, -0.009242285351313074, 66388.959998247, -120.00703926654175, 66385.81676455906, 119.99015075441706]
puVMAGANGLE - LN POLAR
[0.999854602114433, -0.009242285351313074, 0.9999047980749705, -120.00703926654175, 0.9998574568536872, 119.99015075441706]
VOTLAGES - LN CARTESIAN
[66385.6263596549, -10.708553275833136, -33201.54343924919, -57490.447231696584, -33183.0249558669, 57497.508835391505]
puVOTLAGES - LN CARTESIAN
[0.9998545891061392, -0.00016128485521131416, -0.5000587837085627, -0.8658815265665738, -0.49977987106423893, 0.8659878835785958]
VLL - LL CARTESIAN
[99587.16979890408, 57479.7386